In [1]:
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm
import ast
from time import sleep
from bs4 import BeautifulSoup
#change file location according to where this notebook is located.
data_folder = ''
api = ''

In [2]:
drug_df = pd.read_csv(data_folder+'data/final_UK/drug_data.csv')
print(drug_df.shape)
drug_df.head(1)

(11450, 6)


,ingredient_id,ingredient_name,product_id,product_name,active_ingredients,company_name
0,/emc/ingredient/1524,"2,4-dichlorobenzyl alcohol",/emc/product/1589/smpc,\n Boots Antiseptic Cream\n...,"allantoin, cetrimide, 2,4-dichlorobenzyl alcohol",THE BOOTS COMPANY PLC


In [6]:
drug_df.ingredient_id.nunique(), drug_df.product_id.nunique(), drug_df.shape[0]

(1978, 9334, 11450)

----

In [ ]:
drug_info, drug_content = [], []
for product in tqdm(drug_df.product_id.unique().tolist()):
  product_id = product.split('/')[-2]
  f = (data_folder+'data/raw/{}.txt'.format(product_id))
  with open(f) as fi:
      s = BeautifulSoup(fi, 'html.parser')
  #product header
  try:
    header = s.find(class_='product-header')
    try:
      name = header.find('h2')#.text.replace('\t','').strip()
    except:
      name = ''
    try:
      ingredient = header.find(class_='product-header-item product-header-active-ingredient')#.text.replace('\t','').strip()
    except:
      ingredient = ''
    try:
      company_header = header.find(class_='product-header-item product-header-company')#.text.replace('\t','').strip()
    except:
      company_header = ''
  except:
    pass
  #about medicine
  try:
    about = s.find(class_='product-about-medicine')#.text.replace('\t','').strip()
  except:
    about = ''
  #last updated
  try:
    last_updated = s.find(class_='last-updated')#.text.replace('\t','').strip()
  except:
    last_updated = ''
  #content
  try:
    content = s.find_all('details')
    smpc_sections = []
    for c in content:
      try:
        if c.find('summary')['data-evt'] == 'smpcSectionOpen':
          summary_id = c.find('summary')['id']
          summary_name = c.find('summary').text
          c_text = c.find('div')##.text.replace('\t','').strip()
          smpc_sections.append([summary_id, summary_name, c_text])
      except:
        pass
  except:
    pass
  #company
  try:
    company = s.find(class_='company-details')
    try:
      company_title = company.find(class_="company-details__header")#.text.replace('\t','').strip()
    except:
      company_title = ''
    try:
      contact_items = company.find(class_="company-details-contact-items")
      contact_items = dict(zip([i.text for i in contact_items.find_all(class_='company-contacts-item-title')],
                               [i for i in contact_items.find_all('span')])) ##.text.replace('\t','').strip()
    except:
      contact_items = {}
  except:
    pass

  drug_info.append([product_id, name, ingredient, company_header, about, last_updated, company_title, contact_items])
  drug_content.append([product_id, smpc_sections])

drug_info_df = pd.DataFrame(drug_info, columns = ['product_id', 'drug_name', 'ingredients', 'company_header', 'about', 'last_updated', 'company_title', 'contact_items'])
drug_content_df = pd.DataFrame(drug_content, columns = ['product_id', 'smpc_sections'])
drug_content_df = drug_content_df.explode('smpc_sections')
drug_info_df.to_csv(data_folder+'drug_info_raw_all.csv', index = False)
drug_content_df.to_csv(data_folder + 'drug_content_raw_all.csv', index = False)

 98%|█████████▊| 9132/9334 [1:28:18<01:57,  1.72it/s]

In [10]:
from glob import glob
glob(data_folder+'drug*')
!ls -lh $data_folder

total 1.4G
-rw------- 1 root root  28K Sep 24 21:49 0-1.obtain_data.ipynb
-rw------- 1 root root 118K Dec 16 18:21 1-2.parse_labels.ipynb
-rw------- 1 root root 109K Sep 25 17:24 2.term_extraction_ade.ipynb
-rw------- 1 root root  55K Dec 10 23:51 4.validation.ipynb
-rw------- 1 root root 1.1K Aug  7 00:54 99.full_processing_pipeline.ipynb
-rw------- 1 root root 113K Sep 25 18:57 build_tables.ipynb
drwx------ 5 root root 4.0K Jun  1  2023 data
-rw------- 1 root root 448M Dec 16 14:17 drug_content_all.csv
-rw------- 1 root root 450M Dec 16 17:30 drug_content_all_formatted.csv
-rw------- 1 root root 448M Jan 21 00:14 drug_content_raw_all.csv
-rw------- 1 root root 3.7M Jan 21 00:16 drug_info_all.csv
-rw------- 1 root root 3.6M Dec 16 18:21 drug_info_all_formatted.csv
-rw------- 1 root root  20M Jan 21 00:14 drug_info_raw_all.csv
-rw------- 1 root root 3.1K Dec 16 17:23 DRUGLABEL_FORMAT.md
drwx------ 2 root root 4.0K Sep 24 21:59 notebooks
-rw------- 1 root root  62K Jul  9  2023 parse_te

In [ ]:
drug_info_df = pd.read_csv(data_folder+'drug_info_raw_all.csv')
drug_info_df['drug_name'] = drug_info_df['drug_name'].apply(lambda x: BeautifulSoup(x).text.replace('\t','').strip() if str(x) != 'nan' else None)
drug_info_df['ingredients'] = drug_info_df['ingredients'].apply(lambda x: BeautifulSoup(x).find(class_="product-header-active-ingredient-items").text.replace('\t','').strip() if str(x) != 'nan' else None)
drug_info_df['company_header'] = drug_info_df['company_header'].apply(lambda x: BeautifulSoup(x).find(class_="product-header-company-name").text.replace('\t','').strip() if str(x) != 'nan' else None)
drug_info_df['about'] = drug_info_df['about'].apply(lambda x: BeautifulSoup(x).find(class_="product-about-medicine-item-text").text.replace('\t','').strip() if str(x) != 'nan' else None)
drug_info_df['last_updated'] = drug_info_df['last_updated'].apply(lambda x: BeautifulSoup(x).find('span').text.replace('\t','').strip() if str(x) != 'nan' else None)
drug_info_df['company_title'] = drug_info_df['company_title'].apply(lambda x: BeautifulSoup(x).text.replace('\t','').strip() if str(x) != 'nan' else None)
drug_info_df['contact_items'] = drug_info_df['contact_items'].apply(lambda x: {i.split(':')[0].replace("'",'').replace('{',''):BeautifulSoup(i.split(': ')[-1].replace('}','')).text for i in x.split(", '")}
                                                                    if str(x) != 'nan' else None)
drug_info_df.to_csv(data_folder+'drug_info_all.csv', index=False)
drug_info_df.head()

In [ ]:
drug_content_df = pd.read_csv(data_folder + 'drug_content_raw_all.csv')
drug_content_df['smpc_sections'] = drug_content_df['smpc_sections'].apply(lambda x: x.replace('[','').replace(']','').split("', ") if str(x) != 'nan' else None)
drug_content_df['section_id'] = drug_content_df['smpc_sections'].apply(lambda x: x[0].replace("'",'') if x != None else None)
drug_content_df['section_title'] = drug_content_df['smpc_sections'].apply(lambda x: x[1].replace("'",'') if x != None else None)
drug_content_df['section_content'] = drug_content_df['smpc_sections'].apply(lambda x: BeautifulSoup(x[2]) if x != None else None)
drug_content_df.to_csv(data_folder + 'drug_content_all.csv', index=False)
drug_content_df.head()

----

In [ ]:
import xml.etree.ElementTree as ET
#format into individual drug label XML files
def drug_label_xml(drug_id, out_folder):
    drug_info = drug_info_df[drug_info_df.product_id == drug_id]
    drug_content = drug_content_df[drug_content_df.product_id == drug_id]

    root = ET.Element("drug_label")

    drug_i = ET.Element("drug_info")
    root.append(drug_i)

    for col in drug_info.columns:
      subelement = ET.SubElement(drug_i, col)
      if str(drug_info[col].iloc[0]) != 'nan':
        subelement.text = str(drug_info[col].iloc[0])
      else:
        subelement.text = ''

    drug_label = ET.Element("drug_label_info")
    root.append(drug_label)

    for i, row in drug_content.iterrows():
      subelement = ET.SubElement(drug_label, row['section_id'])
      subelement_title = ET.SubElement(subelement, 'title')
      if str(row['section_title']) != 'nan':
        subelement_title.text = row['section_title']
      else:
        subelement_title.text = ''
      subelement_content = ET.SubElement(subelement, 'content')
      if row['section_content'] != None:
        subelement_content.text = str(row['section_content'].text)
      else:
        subelement_content.text = ''

    tree = ET.ElementTree(root)

    with open (out_folder+'{}.xml'.format(str(drug_id)), "wb") as f:
        tree.write(f)

In [ ]:
for drug_id in tqdm(drug_info_df.product_id.unique()):
  drug_label_xml(drug_id, data_folder+'xml_data/')

---

In [ ]:
out_file = data_folder+'data/final_UK/drug_label_xml.tar.gz'
xml_dir = data_folder+'xml_data'
!tar -czf $out_file $xml_dir

In [13]:
#also copy over the formatted csv files
fs = [data_folder+'drug_info_all.csv', data_folder +'drug_content_all.csv']
out_folder = data_folder+'data/final_UK'
for f in fs:
  !cp $f $out_folder